<a href="https://colab.research.google.com/github/leakysam/ScheduleScraper/blob/main/Scheduling_Scraper_using_S3_bucket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import boto3
from io import BytesIO

def lambda_handler(event, context):
    # Step 1: Get the webpage content
    url = "http://agromarketday.com/markets/9-nakawa-market"  # Replace with the actual URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Step 2: Parse the table
    data = []
    table = soup.find('table', class_='table-market-prices')
    rows = table.find_all('tr')

    for row in rows[1:]:  # Skip the header row
        cols = row.find_all('td')
        commodity = cols[0].text.strip()
        price = cols[1].text.strip()
        unit = cols[2].text.strip()
        data.append([commodity, price, unit])

    # Step 3: Create a DataFrame
    df = pd.DataFrame(data, columns=['Commodity', 'Price', 'Units'])

    # Step 4: Save to Excel and upload to S3
    s3 = boto3.client('s3')
    bucket = 'your-s3-bucket-name'
    file_name = 'commodity_prices.xlsx'

    # Save the DataFrame to a buffer
    buffer = BytesIO()
    df.to_excel(buffer, index=False)
    buffer.seek(0)

    # Upload to S3
    s3.put_object(Bucket=bucket, Key=file_name, Body=buffer.getvalue())

    return {
        'statusCode': 200,
        'body': 'Data successfully saved to S3.'
    }
